In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="7"

Fri May 19 17:49:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   27C    P8    13W / 230W |   1433MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 8
n_outputs = 61
n_frames = 400
sr = 24000
fps = 30
train_csv_files = ['/Storage/speech/face/rvh_visme_1_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_2_iPhone_30fps.csv',
                  '/Storage/speech/face/rvh_visme_3_iPhone_30fps.csv']
train_wav_files = ['/Storage/speech/face/rvh_visme_1_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_2_iPhone.wav',
                 '/Storage/speech/face/rvh_visme_3_iPhone.wav']

test_csv_files = ['/Storage/speech/face/rvh_visme_4_iPhone_30fps.csv']
test_wav_files = ['/Storage/speech/face/rvh_visme_4_iPhone.wav']

### Model

In [8]:
from model.model_vqvae import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, out_dim=n_outputs, K=8, latent_dim=256)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [9]:
save_dir = '/data/scpark/save/lips/train05.19-3/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 2207, model, None, optimizer)

total 0
-rw-rw-r-- 1 scpark scpark 0  5월 19 17:50 events.out.tfevents.1684486205.GPUSVR01
-rw-rw-r-- 1 scpark scpark 0  5월 19 17:49 events.out.tfevents.1684486191.GPUSVR01


In [10]:
import torch
from data.dataset import LipsDataset, Collate

dataset = LipsDataset(train_wav_files, train_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps, perturb=True)
print('train dataset :', len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8, 
                                           collate_fn=Collate(n_frames, n_mels))
print(train_loader)

dataset = LipsDataset(test_wav_files, test_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps, perturb=False)
print('test dataset :', len(dataset))
test_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8,
                                           collate_fn=Collate(n_frames, n_mels))
print(test_loader)

train dataset : 22042
test dataset : 7725


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = batch['inputs'].transpose(1, 2).to(device)
        targets = batch['outputs'].transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = batch['inputs'].transpose(1, 2).to(device)
                targets = batch['outputs'].transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 2.061098575592041
test : 1 2.0616862773895264
test : 2 2.060452461242676
test : 3 2.0599799156188965
test : 4 2.0610709190368652
test : 5 2.0619049072265625
test : 6 2.0609099864959717
test : 7 2.061115264892578
test : 8 2.061785936355591
test : 9 2.0606460571289062
test_loss : 2.061065196990967
saved /data/scpark/save/lips/train05.19-3/save_0
1
auto_encoding_loss 0.08060593158006668
commit_loss 0.0005267133237794042
zi_prediction_loss 2.044222593307495
1 2.1253552436828613
2
auto_encoding_loss 0.07014196366071701
commit_loss 0.0005122608854435384
zi_prediction_loss 2.027386426925659
2 2.0980405807495117
3
auto_encoding_loss 0.06299041956663132
commit_loss 0.000519749999511987
zi_prediction_loss 1.9960893392562866
3 2.0595993995666504
4
auto_encoding_loss 0.05753957852721214
commit_loss 0.0005416167550720274
zi_prediction_loss 1.9759162664413452
4 2.0339975357055664
5
auto_encoding_loss 0.057844214141368866
commit_loss 0.0005728437099605799
zi_prediction_loss 1.963767051696777

In [ ]:
save(save_dir, step, model, None, optimizer)